In [86]:
import os
from dotenv import load_dotenv

from langgraph.graph import StateGraph
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from typing import TypedDict,Literal,Dict
from pydantic import BaseModel,Field

In [3]:
load_dotenv()

True

In [14]:
parser = StrOutputParser()

In [4]:
model = ChatOpenAI(api_key=os.getenv('OPEN_AI_API_KEY'))

In [74]:
class SupportState(TypedDict):
    review: str
    sentiment: bool
    response: str

In [75]:
class Diagnosis(BaseModel):
    issue_type: str = Field(description="Which segment of products is the issue concerned to.")
    tone: str = Field(description='Tone of the customer')
    urgency: str = Field(description='Urgency of the issue')

In [76]:
structured_model = model.with_structured_output(Diagnosis)

c:\Users\ZXGAMING\miniconda3\envs\langchain\lib\site-packages\langchain_openai\chat_models\base.py:1673: UserWarning: Cannot use method='json_schema' with model gpt-3.5-turbo since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


In [89]:
def find_sentiment(state: SupportState):
    review = state['review']

    prompt = f"Find the sentiment of the following response and categorise only as Positive or Negative \n {review}"

    sentiment = model.invoke(prompt).content

    return{
        'sentiment':sentiment
    }

In [90]:
def positive_response(state: SupportState):
    review = state['review']

    prompt = f'Generate a positive response based upon the review context\n review: {review} \n'

    response = model.invoke(prompt).content

    return{
        'response':response
    }

In [91]:
def combine_inputs(review: str, analysis: Diagnosis) -> Dict:
    return{
        'review': review,
        'analysis': analysis
    }

In [92]:
def negative_response(state: SupportState):
    review = state['review']

    analysis_prompt = PromptTemplate(template='Analyse the following customer review and provide the category/type of issue, tone of customer and urgency of the issue \n {review}',input_variables=['review'])

    response_prompt = PromptTemplate(template='Based upon the negative customer review and the analysis of the review, generate a response \n {review} \n {analysis}',input_variables=['review','analysis'])

    analysis_chain = analysis_prompt | structured_model

    final_chain = analysis_chain | RunnableLambda(lambda analysis:combine_inputs(review,analysis)) | response_prompt | model | parser

    response = final_chain.invoke({'review':review})

    return{
        'response': response
    }

In [93]:
def generate_response(state: SupportState) -> Literal['positive_response','negative_response']:
    sentiment = state['sentiment']

    if sentiment == 'Positive':
        return 'positive_response'
    else:
        return 'negative_response'

In [94]:
graph = StateGraph(SupportState)

graph.add_node('find_sentiment',find_sentiment)
graph.add_node('positive_response',positive_response)
graph.add_node('negative_response',negative_response)

graph.set_entry_point('find_sentiment')

graph.add_conditional_edges('find_sentiment',generate_response)

graph.set_finish_point('positive_response')
graph.set_finish_point('negative_response')

app = graph.compile()

In [97]:
initial_state={
    'review': 'The laptop I brought is overpriced and a crow stole it'
}

app.invoke(initial_state)

{'review': 'The laptop I brought is overpriced and a crow stole it',
 'sentiment': 'Negative',
 'response': "Dear valued customer,\n\nWe are sorry to hear about your experience with our laptop. We understand your frustration regarding the pricing and the unfortunate incident with the crow. Our team is always looking for ways to improve our products and pricing to better meet our customers' needs. \n\nPlease reach out to our customer service team so we can discuss possible solutions and address any concerns you may have. We apologize for any inconvenience this may have caused you and we appreciate your feedback.\n\nSincerely,\n[Your Company Name]"}